In [1]:
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from openpilot.selfdrive.car.ford.values import CAR

database = get_comma_car_segments_database()

platforms = [c.value for c in CAR]

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [2]:
import random

from tqdm.contrib.concurrent import thread_map

from openpilot.tools.lib.logreader import LogReader


def process(segment):
  CP = LogReader(segment).first("carParams")
  return CP.carVin, CP.carFw, CP.carFingerprint, CP.fuzzyFingerprint, CP.fingerprintSource


FW_TO_CHECK = set(thread_map(process, [
  segment for platform in platforms
  for segment in random.sample(database[platform], min(len(database[platform]), 50))
]))

  0%|          | 0/261 [00:00<?, ?it/s]

In [3]:
import pandas as pd

from openpilot.selfdrive.car.fw_versions import build_fw_dict
from openpilot.selfdrive.car.ford.fingerprints import FW_VERSIONS
from openpilot.selfdrive.car.ford.values import match_fw_to_car_fuzzy

rows = []
success, count = 0, 0
for vin, car_fw, expected_platform, fuzzy, source in FW_TO_CHECK:
  car_fw = list(filter(lambda fw: fw.brand == "ford" and not fw.logging, car_fw))
  if len(car_fw) == 0:
    print(f"Skipping vin: {vin} platform: {expected_platform}, no car fw")
    continue

  fuzzy_fingerprint = match_fw_to_car_fuzzy(build_fw_dict(car_fw), FW_VERSIONS)
  if len(fuzzy_fingerprint) == 0:
    fuzzy_fingerprint = "mock"
  elif len(fuzzy_fingerprint) > 1:
    fuzzy_fingerprint = "multiple"
  else:
    fuzzy_fingerprint = list(fuzzy_fingerprint)[0]

  correct = fuzzy_fingerprint == expected_platform
  rows.append((source, expected_platform, fuzzy_fingerprint, correct))
  if not correct:
    print(f"vin: {vin}   expected: {expected_platform: <30}    fuzzy: {fuzzy_fingerprint: <30}    correct: {correct}")
    print(f"  source: {source}")
    print(f"  fuzzy: {fuzzy}")
    print("  car_fw:")
    fws = set()
    for fw in car_fw:
      fw_version = fw.fwVersion.rstrip(b"\0")
      fws.add(f"{fw.ecu} {fw_version}")
    for fw in sorted(fws):
      print(f"    {fw}")
  success += correct
  count += 1

print(f"Success rate: {success / count * 100:.2f}%")

pd.set_option("display.max_rows", None)

df = pd.DataFrame(rows, columns=["source", "expected", "fuzzy", "correct"]).drop_duplicates()
df

vin: 3FTTW8E3XPRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: mock                              correct: False
  source: fw
  fuzzy: True
  car_fw:
    abs b'PZ6C-2D053-ED'
    engine b'PZ6A-14C204-JE'
    eps b'NZ6C-14D003-AL'
    fwdCamera b'NZ6T-14F397-AC'
vin: 3FTTW8E3XPRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: mock                              correct: False
  source: fw
  fuzzy: True
  car_fw:
    abs b'PZ6C-2D053-ED'
    engine b'PZ6A-14C204-JE'
    eps b'NZ6C-14D003-AL'
    fwdCamera b'NZ6T-14F397-AC'
vin: 3FTTW8E3XPRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: mock                              correct: False
  source: fw
  fuzzy: True
  car_fw:
    abs b'PZ6C-2D053-ED'
    engine b'PZ6A-14C204-JE'
    eps b'NZ6C-14D003-AL'
    fwdCamera b'NZ6T-14F397-AC'
Skipping vin: 00000000000XXXXXX platform: FORD F-150 14TH GEN, no car fw
vin: 3FTTW8E3XPRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: mock                         

,source,expected,fuzzy,correct
0,fw,FORD BRONCO SPORT 1ST GEN,FORD BRONCO SPORT 1ST GEN,True
1,fw,FORD MAVERICK 1ST GEN,mock,False
4,fw,FORD ESCAPE 4TH GEN,FORD ESCAPE 4TH GEN,True
6,fw,FORD FOCUS 4TH GEN,FORD FOCUS 4TH GEN,True
7,fixed,FORD EXPLORER 6TH GEN,FORD EXPLORER 6TH GEN,True
10,fw,FORD EXPLORER 6TH GEN,FORD EXPLORER 6TH GEN,True
19,fw,FORD MAVERICK 1ST GEN,FORD MAVERICK 1ST GEN,True
41,fw,FORD F-150 LIGHTNING 1ST GEN,FORD F-150 LIGHTNING 1ST GEN,True
52,fixed,FORD MAVERICK 1ST GEN,FORD MAVERICK 1ST GEN,True
113,fw,FORD MUSTANG MACH-E 1ST GEN,FORD MUSTANG MACH-E 1ST GEN,True
